### 0. Load and setup

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import os
import json
from mendeleev import element
import ast
from ase import Atoms

PROJECT_ROOT = Path("..").resolve()
DATA_PROCESSED = PROJECT_ROOT / "data_processed"

INPUT_FILE = DATA_PROCESSED / "mamun_HER_processed.csv"
OUTPUT_FILE = DATA_PROCESSED / "mamun_HER_features.csv"

CACHE_FILE = DATA_PROCESSED / "mp_element_vol_cache.json"

df = pd.read_csv(INPUT_FILE)


### 1. Vergard volume per atom

In [ ]:
def load_element_volume_cache() -> dict:
    """
    Load per-element volumes [Å³/atom] from JSON cache.
    This file is part of the repository for full reproducibility.
    """
    if CACHE_FILE.exists():
        with open(CACHE_FILE, "r") as f:
            return json.load(f)
    raise FileNotFoundError(
        f"Element volume cache not found at {CACHE_FILE}. ")

# load cache and sanity-check coverage
ELEMENT_V_ATOM = load_element_volume_cache()

surf_elements = surf_elements = set(df["surf_A"].dropna()) | set(df["surf_B"].dropna())
missing = surf_elements.difference(ELEMENT_V_ATOM.keys())
assert not missing, f"Missing element volumes for: {missing}. Update mp_element_vol_cache.json."


def vegard_vol_per_atom_surface(row):
    """
    Vegard-style volume per atom based on surface composition:
    v = (n v_A + m v_B) / (n + m).
    """
    A, B = row["surf_A"], row["surf_B"]
    n, m = row["surf_n"], row["surf_m"]

    vA = ELEMENT_V_ATOM.get(A)
    vB = ELEMENT_V_ATOM.get(B)

    # pure single-element surface
    if m == 0 or B == A:
        return vA

    return (n * vA + m * vB) / (n + m)

df["vol_per_atom"] = df.apply(vegard_vol_per_atom_surface, axis=1)

### 3. GCN
Assign generalized coordination number (GCN) for H adsorption based on (facet, norm_label) using the discrete values from mapping by Martinez-Alonso (2024).

In [ ]:
def get_gcn_from_row(row) -> float | None:

    facet = str(row["facet"])
    nl = row.get("norm_label")

    if not isinstance(nl, str):
        return None

    nl = nl.lower()

    if nl.startswith("fcc"):
        site_family = "fcc"
    elif nl.startswith("hcp"):
        site_family = "hcp"
    elif nl.startswith("ontop"):
        site_family = "ontop"
    elif nl.startswith("bridge"):
        site_family = "bridge"
    else:
        return None

    # fcc(111) and fcc(101) share the same discrete GCN mapping
    if facet in ("111", "101"):
        if site_family == "fcc":
            return 5.25
        elif site_family == "hcp":
            return 3.25
        elif site_family == "ontop":
            return 0.75
        elif site_family == "bridge":
            return 0.0

    return None

df["GCN"] = df.apply(get_gcn_from_row, axis=1)


### 4. WAR
Calculate weighted atomic radius (WAR) based on surface composition. Uses mendeleev.element(...).atomic_radius (pm).

In [ ]:
_war_cache = {}

def compute_WAR(elem_A: str, elem_B: str, n: int, m: int) -> float | None:

    def r(sym):
        if sym not in _war_cache:
            r_val = element(sym).atomic_radius
            if r_val is None:
                raise ValueError(f"No atomic radius for {sym}")
            _war_cache[sym] = float(r_val)
        return _war_cache[sym]

    if m == 0 or pd.isna(elem_B):
        return r(elem_A)
    
    r_A = r(elem_A)
    r_B = r(elem_B)

    return (n * r_A + m * r_B) / (n + m)

df["WAR"] = df.apply(lambda row: compute_WAR(row["surf_A"], row["surf_B"], row["surf_n"], row["surf_m"]), axis=1)


### 5. WIE
Weighted first ionization energy (WIE) based on surface composition. Uses element(...).ionenergies[1].

In [ ]:
_wie_cache = {}

def compute_WIE(elem_A: str, elem_B: str, n: int, m: int) -> float | None:

    def IE(sym):
        if sym not in _wie_cache:
            e = element(sym)
            val = e.ionenergies.get(1)
            if val is None:
                raise ValueError(f"No first ionization energy for {sym}")
            _wie_cache[sym] = float(val)
        return _wie_cache[sym]

    if m == 0 or pd.isna(elem_B):
        return IE(elem_A)
    
    IE_A = IE(elem_A)
    IE_B = IE(elem_B)

    return (n * IE_A + m * IE_B) / (n + m)

df["WIE"] = df.apply(lambda row: compute_WIE(row["surf_A"], row["surf_B"], row["surf_n"], row["surf_m"]), axis=1)


### 6. WEN
Weighted Pauling electronegativity based on surface composition. Uses element(...).en_pauling.

In [ ]:
_wen_cache = {}

def compute_WEN(elem_A: str, elem_B: str, n: int, m: int) -> float | None:

    def chi(sym):
        if sym not in _wen_cache:
            val = element(sym).en_pauling
            if val is None:
                raise ValueError(f"No Pauling EN for {sym}")
            _wen_cache[sym] = float(val)
        return _wen_cache[sym]

    if m == 0 or pd.isna(elem_B):
        return chi(elem_A)
    
    chi_A = chi(elem_A)
    chi_B = chi(elem_B)

    return (n * chi_A + m * chi_B) / (n + m)

df["WEN"] = df.apply(
    lambda row: compute_WEN(row["surf_A"], row["surf_B"], row["surf_n"], row["surf_m"]),
    axis=1,
)

assert df["WEN"].notna().all(), "WEN has missing values."
print("WEN range:", df["WEN"].min(), "→", df["WEN"].max())

### 7. SA and SB
Get number of valence electrons for element symbol 'sym' using mendeleev.element(...).nvalence().

In [ ]:
_valence_cache = {}

def valence_electrons(sym: str) -> int:

    if pd.isna(sym):
        return 0  # pure A, SB = 0
    
    if sym in _valence_cache:
        return _valence_cache[sym]

    e = element(sym)
    nval = e.nvalence()
    if nval is None:
        raise ValueError(f"Missing valence electron data for element {sym}")
    _valence_cache[sym] = int(nval)
    return _valence_cache[sym]


df["SA"] = df["surf_A"].apply(valence_electrons)
df["SB"] = df["surf_B"].apply(valence_electrons)

pure_mask = df["surf_m"] == 0
df.loc[pure_mask, "SB"] = 0

### 8. PSI
Compute PSI, descriptor of the local electrochemical environment of the adsorption sites that incorporates electronegativities and valence electrone counts of first-layer neighbours. The coefficient mappings are based on the ones defined by Martinez-Alonso (2024).

In [ ]:
missing_eng_pauling = {
    '36': 3.00,  '61': 1.13, '63': 1.20, '65': 1.10, '70': 1.10,
    '86': 2.20,  '95': 1.13, '96': 1.28, '97': 1.30, '98': 1.30,
    '99': 1.30, '100': 1.30, '101': 1.30, '102': 1.30, '103': 1.30,
}

PGtogeom = {'Fm-3m': 'fcc', 'Im-3m': 'bcc', 'P6_3/mmc': 'hcp'}

_en_cache = {}

def get_en_by_Z(Z) -> float:
    
    Z_int = int(Z)

    if Z_int in _en_cache:
        return _en_cache[Z_int]

    el = element(Z_int)
    val = el.en_pauling

    if val is None:
        raise ValueError(f"No Pauling EN available for Z={Z_int}")

    _en_cache[Z_int] = float(val)
    return _en_cache[Z_int]


In [ ]:
def get_psi(sample_info, features):
    """
    Compute Psi for one adsorption configuration.

    sample_info:
      - system: chemical formula
      - Stoich: 'A', 'AB', 'A3B', 'AB3'
      - Geom: 'fcc' / 'bcc' / 'hcp'
      - Type: 'Pure' or 'Alloy'
      - Point_group: 'Fm-3m'
      - site: normalized site label

    features:
      - out_eA, out_eB: outer electrons (SA/SB)
    """

    # build outer-electron zero dict
    outere_zero = {
        k.replace("out_e", "outere") + "_zero": v
        for k, v in features.items()
        if k.startswith("out_e")
    }

    # electronegativities for unique species in the system
    atoms = Atoms(sample_info["system"])
    nums = list(atoms.get_atomic_numbers())
    n_unique = len(set(nums))
    letters = {i: c for i, c in enumerate("ABCD")}

    eng_zero = {}
    seen, idx = [], 0
    for Z in nums:
        if Z in seen:
            continue
        L = letters[idx]
        val = get_en_by_Z(Z)
        eng_zero[f"eng{L}_zero"] = val
        seen.append(Z)
        idx += 1
        if idx == n_unique:
            break

    if sample_info["Type"] == "Pure":
        eng_zero["engB_zero"] = eng_zero["engA_zero"]

    stoich = sample_info["Stoich"]
    geom = sample_info["Geom"]
    site = sample_info["site"]

    P = {}

    # A3B fcc 
    if stoich == "A3B":
        if geom == "fcc" and site == "fccAAA":
            P["A"], P["B"], n = range(5), 0, 6
        elif geom == "fcc" and site == "fccAAB":
            P["A"], P["B"], n = range(3), range(1), 6
        elif geom == "fcc" and site == "hcpAAA":
            P["A"], P["B"], n = range(2), 0, 3
        elif geom == "fcc" and site == "hcpAAB":
            P["A"], P["B"], n = range(1), range(0), 3
        elif geom == "fcc" and site == "ontopA":
            P["A"], P["B"], n = range(6), range(2), 10
        elif geom == "fcc" and site == "ontopB":
            P["A"], P["B"], n = range(8), range(0), 10
        elif geom == "fcc" and site == "bridgeAA":
            P["A"], P["B"], n = range(1), 0, 2
        elif geom == "fcc" and site == "bridgeAB":
            P["A"], P["B"], n = range(0), range(0), 2
        elif geom == "fcc" and site == "bridgeBB":
            P["A"], P["B"], n = 0, range(1), 2
        else:
            raise ValueError(f"A3B: unknown site {geom}/{site}")

    # Pure A 
    elif stoich == "A":
        if geom == "fcc" and site == "fccAAA":
            P["A"], P["B"], n = range(5), 0, 6
        elif geom == "fcc" and site == "hcpAAA":
            P["A"], P["B"], n = range(2), 0, 3
        elif geom == "fcc" and site == "ontopA":
            P["A"], P["B"], n = range(9), 0, 10
        elif site.startswith("bridge"):
            P["A"], P["B"], n = range(1), 0, 2
        elif geom in ("bcc", "fcc", "hcp") and site.startswith(
            ("fcc", "hcp", "ontop", "hollow", "shortbridge", "longbridge", "threefold", "top")
        ):
            P["A"], P["B"], n = range(0), 0, 1
        else:
            raise ValueError(f"Pure-A: unknown site {geom}/{site!r}")

    # AB fcc
    elif stoich == "AB":
        if geom == "fcc" and site == "fccAAB":
            P["A"], P["B"], n = range(3), range(1), 6
        elif geom == "fcc" and site == "fccABB":
            P["A"], P["B"], n = range(1), range(3), 6
        elif geom == "fcc" and site == "hcpAAB":
            P["A"], P["B"], n = range(1), range(0), 3
        elif geom == "fcc" and site == "hcpABB":
            P["A"], P["B"], n = range(0), range(1), 3
        elif geom == "fcc" and site == "ontopA":
            P["A"], P["B"], n = range(3), range(5), 10
        elif geom == "fcc" and site == "ontopB":
            P["A"], P["B"], n = range(5), range(3), 10
        elif geom == "fcc" and site == "bridgeAA":
            P["A"], P["B"], n = range(1), 0, 2
        elif geom == "fcc" and site == "bridgeAB":
            P["A"], P["B"], n = range(0), range(0), 2
        elif geom == "fcc" and site == "bridgeBB":
            P["A"], P["B"], n = 0, range(1), 2
        elif site in ("fccAAA", "hcpAAA"):
            count = 6 if site == "fccAAA" else 3
            rng = range(5) if site == "fccAAA" else range(2)
            P["A"], P["B"], n = rng, 0, count
        else:
            raise ValueError(f"AB: unknown site {geom}/{site}")

    # AB3 fcc
    elif stoich == "AB3":
        if geom == "fcc" and site == "fccAAB":
            P["A"], P["B"], n = range(3), range(1), 6
        elif geom == "fcc" and site == "fccABB":
            P["A"], P["B"], n = range(1), range(3), 6
        elif geom == "fcc" and site == "hcpAAB":
            P["A"], P["B"], n = range(1), range(0), 3
        elif geom == "fcc" and site == "hcpABB":
            P["A"], P["B"], n = range(0), range(1), 3
        elif geom == "fcc" and site == "ontopA":
            P["A"], P["B"], n = range(3), range(5), 10
        elif geom == "fcc" and site == "ontopB":
            P["A"], P["B"], n = range(5), range(3), 10
        elif geom == "fcc" and site == "bridgeAA":
            P["A"], P["B"], n = range(1), 0, 2
        elif geom == "fcc" and site == "bridgeAB":
            P["A"], P["B"], n = range(0), range(0), 2
        elif geom == "fcc" and site == "bridgeBB":
            P["A"], P["B"], n = 0, range(1), 2
        elif site == "fccAAA":
            P["A"], P["B"], n = range(5), 0, 6
        elif site == "hcpAAA":
            P["A"], P["B"], n = range(2), 0, 3
        else:
            raise ValueError(f"AB3: unknown site {geom}/{site}")

    else:
        raise ValueError(f"Unsupported stoichiometry {stoich}")

    # robust zero-neighbor test and products
    outere_dict = {}
    eng_dict = {}

    for L, rng in P.items():
        if rng == 0:
            outere_dict[L] = 1.0
            eng_dict[L] = 1.0
        else:
            oe = outere_zero[f"outere{L}_zero"]
            ee = eng_zero[f"eng{L}_zero"]
            outere_dict[L] = oe ** (len(rng) + 1)
            eng_dict[L] = ee ** (len(rng) + 1)

    # geometric means
    p_e = np.prod(list(outere_dict.values())) ** (1.0 / n)
    p_eng = np.prod(list(eng_dict.values())) ** (1.0 / n)

    return (p_e * p_e) / p_eng


In [ ]:
def make_type_and_geom(n, m, point_group="Fm-3m"):
    if m == 0:
        return "A", PGtogeom[point_group]
    if n == 1 and m == 1:
        return "AB", "fcc"
    if n == 3 and m == 1:
        return "A3B", "fcc"
    if n == 1 and m == 3:
        return "AB3", "fcc"


def normalize_site(raw_site: str) -> str:
    """
    Normalize Mamun 'sites' encodings for H:
    - 'hollow|A_A_B|FCC' to 'fccAAB'
    - 'top|A' to 'ontopA'
    - 'bridge|A_B|A' to 'bridgeAB'
    """
    parts = raw_site.split("|")
    pos = parts[0].split("-")[0]  # disregard tilt
    pos = "ontop" if pos == "top" else pos
    neigh = parts[1] if len(parts) > 1 else ""
    code = neigh.replace("_", "")
    plane = parts[2].lower() if len(parts) > 2 else ""

    if pos == "hollow" and plane:
        return f"{plane}{code}"
    return f"{pos}{code}"


In [ ]:
if "surf_comp" not in df.columns:
    def make_surf_comp(row):
        A, B, n, m = row["surf_A"], row["surf_B"], row["surf_n"], row["surf_m"]
        if m == 0 or A == B:
            return f"{A}{n}"
        return f"{A}{n}{B}{m}"
    df["surf_comp"] = df.apply(make_surf_comp, axis=1)

psis = []

for i, row in df.iterrows():
    n = int(row["surf_n"])
    m = int(row["surf_m"])
    stoich, geom = make_type_and_geom(n, m)

    sites_dict = ast.literal_eval(row["sites"])
    raw_site = sites_dict["H"]
    site = normalize_site(raw_site)

    sample_info = {
        "system": row["surf_comp"],
        "Stoich": stoich,
        "Geom": geom,
        "Type": "Pure" if m == 0 else "Alloy",
        "Point_group": "Fm-3m",
        "site": site,
    }

    feats = {
        "out_eA": int(row["SA"]),
        "out_eB": int(row["SB"]),
    }

    psi = get_psi(sample_info, feats)
    psis.append(psi)

df["Psi"] = psis


### 9. Save feature columns

In [ ]:
df.to_csv(OUTPUT_FILE, index=False)